In [19]:
from pyspark.sql import SparkSession

In [20]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('test')\
  .getOrCreate()

### Green Taxi

In [21]:
df_green = spark.read.parquet('./data/pq/green/*/*')

In [22]:
df_green.createOrReplaceTempView('green_trip_data')

In [23]:
df_green_revenue = spark.sql("""
  SELECT 
    date_trunc("hour", lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(*) AS row_count
  FROM green_trip_data
  WHERE lpep_pickup_datetime >= '2020-01-01 00:00:00'
  GROUP BY 1, 2
  ORDER BY 1, 2
""")

In [24]:
df_green_revenue.show()

+-------------------+----+------------------+---------+
|               hour|zone|            amount|row_count|
+-------------------+----+------------------+---------+
|2020-01-01 00:00:00|   7| 769.7299999999997|       45|
|2020-01-01 00:00:00|  17|            195.03|        9|
|2020-01-01 00:00:00|  18|               7.8|        1|
|2020-01-01 00:00:00|  22|              15.8|        1|
|2020-01-01 00:00:00|  24|              87.6|        3|
|2020-01-01 00:00:00|  25|             531.0|       26|
|2020-01-01 00:00:00|  29|              61.3|        1|
|2020-01-01 00:00:00|  32| 68.94999999999999|        2|
|2020-01-01 00:00:00|  33|317.27000000000004|       11|
|2020-01-01 00:00:00|  35|129.95999999999998|        5|
|2020-01-01 00:00:00|  36|295.34000000000003|       11|
|2020-01-01 00:00:00|  37|175.67000000000002|        6|
|2020-01-01 00:00:00|  38| 98.78999999999999|        2|
|2020-01-01 00:00:00|  40|168.98000000000002|        8|
|2020-01-01 00:00:00|  41|1363.9599999999987|   

In [25]:
df_green_revenue\
  .repartition(20)\
  .write.parquet('data/report/revenue/green', mode='overwrite')

25/03/14 10:51:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/14 10:51:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/14 10:51:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### Yellow Taxi

In [26]:
df_yellow = spark.read.parquet('./data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow_trip_data')

In [27]:
df_yellow_revenue = spark.sql("""
  SELECT 
    date_trunc("hour", tpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(*) AS row_count
  FROM yellow_trip_data
  WHERE tpep_pickup_datetime >= '2020-01-01 00:00:00'
  GROUP BY 1, 2
  ORDER BY 1, 2
""")

In [28]:
df_yellow_revenue\
  .repartition(20)\
  .write.parquet('data/report/revenue/yellow', mode='overwrite')

25/03/14 10:52:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/14 10:52:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [29]:
df_green_revenue_tmp = df_green_revenue\
  .withColumnRenamed('amount', 'green_amount')\
  .withColumnRenamed('row_count', 'green_row_count')

In [30]:
df_yellow_revenue_tmp = df_yellow_revenue\
  .withColumnRenamed('amount', 'yellow_amount')\
  .withColumnRenamed('row_count', 'yellow_row_count')

In [31]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [ ]:
spark.stop()